### Index

In [1]:
import bs4, tiktoken, numpy as np, os

from langchain import hub
from langchain.load import dumps, loads
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

learn_api_key = os.environ['LANGCHAIN_API_KEY']
openai_api_key = os.environ['OPENAI_API_KEY']
groq_api_key = os.environ['GROQ_API_KEY']

In [2]:
### INDEXING ###

# Load Documents
loader = WebBaseLoader(
    web_paths=['https://lilianweng.github.io/posts/2023-06-23-agent/'],
    bs_kwargs=dict(
        parse_only=bs4.filter.SoupStrainer(
            class_=('post-content', 'post-title', 'post-header')
        )
    ),
)
blog_docs = loader.load()

In [3]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50,
)
splits = text_splitter.split_documents(blog_docs)

In [4]:
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())
retriever = vectorstore.as_retriever()

### Prompt

In [5]:
# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relavant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternate questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (prompt_perspectives
| ChatGroq(model='gemma2-9b-it', api_key=groq_api_key, temperature=0)
| StrOutputParser()
| (lambda x: x.split('\n'))
)

In [6]:
def get_unique_union(documents: list[list]):
    """Unique union of retrieved docs"""
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]

    # Get unique documents
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

C:\Users\user\AppData\Local\Temp\ipykernel_8652\3051904376.py:8: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


12

In [7]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatGroq(model='gemma2-9b-it', api_key=groq_api_key, temperature=0)

final_rag_chain = (
    {"context":retrieval_chain,
     "question":itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()   
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents is the process of breaking down a large, complex task into smaller, more manageable subtasks. \n\nThis is important because LLMs have limited context length and struggle with complex, multi-step tasks. By decomposing a task, the agent can focus on one subtask at a time, making the problem more tractable.\n\nThere are a few ways to achieve task decomposition:\n\n* **Simple prompting:** Using instructions like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?" to guide the LLM.\n* **Task-specific instructions:** Providing more specific instructions tailored to the type of task, such as "Write a story outline" for writing a novel.\n* **Human input:**  Humans can help break down complex tasks into smaller subtasks.\n\n\nThis approach allows the agent to make progress on the overall goal by tackling smaller, more manageable pieces. \n'